In [5]:
%pip install openai pandas numpy


  Using cached openai-1.52.2-py3-none-any.whl.metadata (24 kB)
  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.6.1-cp313-none-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cache

In [10]:
import openai
import pandas as pd
import numpy as np
from getpass import getpass

openai.api_key = getpass()

In [11]:
df = pd.read_csv("words.csv")
df

,text
0,red
1,potatoes
2,soda
3,cheese
4,water
5,blue
6,crispy
7,hamburger
8,coffee
9,green


In [12]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [36]:
cappuccino = get_embedding("cappuccino", engine="text-embedding-ada-002")

In [37]:
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine="text-embedding-ada-002"))
df.to_csv("word_embeddings.csv", index=False)

In [39]:
import ast

def list_of_float64(value):
    return np.array(ast.literal_eval(value), dtype=np.float64)

df = pd.read_csv("word_embeddings.csv", converters={"embedding": list_of_float64})
df

,text,embedding
0,red,"[-6.423245940823108e-05, -0.024778805673122406..."
1,potatoes,"[0.004971202462911606, -0.031133046373724937, ..."
2,soda,"[0.025859493762254715, -0.007452284451574087, ..."
3,cheese,"[-0.0038699328433722258, -0.009271041490137577..."
4,water,"[0.019031280651688576, -0.01257743313908577, 0..."
5,blue,"[0.005434895399957895, -0.0072994716465473175,..."
6,crispy,"[-0.0010056837927550077, -0.005415474995970726..."
7,hamburger,"[-0.01317964494228363, -0.001876765862107277, ..."
8,coffee,"[-0.0007566261338070035, -0.0194522924721241, ..."
9,green,"[0.01538460049778223, -0.010931522585451603, 0..."


In [45]:
coca = get_embedding("coca-cola", engine="text-embedding-ada-002")

In [47]:
from openai.embeddings_utils import cosine_similarity
df["similarities"] = df["embedding"].apply(lambda x: cosine_similarity(x, cappuccino))
res = df.sort_values("similarities", ascending=False)
res

,text,embedding,similarities
19,espresso,"[-0.02250584401190281, -0.012747502885758877, ...",0.916526
15,latte,"[-0.015634099021553993, -0.003942839801311493,...",0.893022
22,mocha,"[-0.012487593106925488, -0.026140518486499786,...",0.877394
8,coffee,"[-0.0007566261338070035, -0.0194522924721241, ...",0.869637
10,milk,"[0.0009238893981091678, -0.019352708011865616,...",0.845948
13,chocolate,"[0.0015507291536778212, -0.012970025651156902,...",0.822439
3,cheese,"[-0.0038699328433722258, -0.009271041490137577...",0.810949
20,cheesecake,"[0.01136965025216341, -0.012745062820613384, -...",0.806181
7,hamburger,"[-0.01317964494228363, -0.001876765862107277, ...",0.802164
18,cheeseburger,"[-0.018216600641608238, 0.005054354667663574, ...",0.800163
